# Numerical Panel Methods 🚢 

In the first notebook we studied

 1. _How can we calculate the influence of each panel?_ **Gaussian quadratures.**
 2. _How should we compute the derivatives of our functions?_ **Automatic differentiation.**

In this notebook we will actually start solving p-flow problems with panels methods addressing two more questions: 

 3. _How can we determine the correct strength for each panel?_ **Set-up and solve a linear system.**
 4. _How should we determine if a method is working?_ **Convergence and validation tests.** 

## 3D example: sphere

To get started, let's set up a example problem: the 3D example of the flow around a sphere.

The `sphere` function below creates a vector of panels using the parametric equation equation 

$$ [S_x,S_y,S_z] = R[\cos(θ₂)\sin(θ₁),\sin(θ₂)\sin(θ₁),\cos(θ₁)] $$

where `θ₁,θ₂` are the azimuth and polar angles.

In [ ]:
include("../src/solve.jl")
using Plots
"""
    sphere(h;R=1) -> Table(panel_props)

Sample a sphere of radius `R` such that the arc-length ≈ h in each direction.
θ₁=[0,π]: azimuth angle, θ₂=[0,2π]: polar angle.
"""
function sphere(h;R=1)
    S(θ₁,θ₂) = R .* [cos(θ₂)*sin(θ₁),sin(θ₂)*sin(θ₁),cos(θ₁)]
    dθ₁ = π/round(π*R/h)
    # dθ₂ = 2π/round(2π*a/h) evenly space polars?
    mapreduce(vcat,0.5dθ₁:dθ₁:π) do θ₁
        dθ₂ = 2π/round(2π*R*sin(θ₁)/h) # get polar step at this azimuth
        param_props.(S,θ₁,0.5dθ₂:dθ₂:2π,dθ₁,dθ₂)
    end |> Table
end

# Create the panels and check how many of them were made
h = 0.5; panels = sphere(h); N=length(panels)

# Split the centroids (`panels.x`) into  `x,y,z` vectors for plotting
x,y,z = eachrow(stack(panels.x));
plot(x,y,z,legend=false,title="Sphere represented with $N panels")

The function sets the azimuth and polar spacings so each panel's arc length is approximately $h$. Then, it evaluates the function `S(θ₁,θ₂)` at those spacings, filling a vector with all the panel information using the `param_props` function defined in `solve.jl`. 

I've set $h=1/2$ and left the $R=1$ default. Let's check if the area's add up to 4π and they are all approximately $h^2$

In [ ]:
# Check the areas `panels.dA`
A_error = sum(panels.dA)/4π-1
A_percent = round(100A_error,digits=1)
plot(z,panels.dA/h^2,ylim=(0,2),xlabel="z",ylabel="dA/h²",
    legend=false, title="Error in total surface area=$A_percent%")

Looks good. 

Note that the number of panels is approximately $N\approx 4\pi R^2/h^2$. This shows that $N \sim 1/h^2$, which we might worry us as we start thinking about how to solve for the flow on these panels.

## Total flow equations

The potential of the full sphere is simply the superposition of each panel's contribution,

$$ \phi(x) = \sum_{i=1}^N q_i \varphi_i(x) $$

where $N$ is the number of panels, $q$ is the vector of **unknown** panel strengths and $\varphi_i$ is influence of panel $i$.

Since we used a $G$ satisfying $\nabla^2 G=0$ and this equation is linear, _any_ vector $q$ will satisfy the laplace equation $\nabla^2\phi=0$ and be a valid potential flow. This is nice since we can't mess that up, but it means we need an additional equation to determine the _correct_ $q$ for a given geometry and flow condition.

 3. _How can we determine the correct $q$ for each panel?_ 

## Apply boundary conditions

The additional equations from our problem description are the boundary conditions. Defining $\vec U$ as the free stream velocity and $\hat n$ as the surface normal, the conditions in an infinite fluid (no free surface) are
 - Flow tangency on the solid body's surface: $U_n+u_n = U_n+\frac{\partial\phi}{\partial n}=0$
 - No disturbance far from the body: $u(\infty)\rightarrow 0$ 

The second condition is achieved automatically since $u(r) \sim \frac{\partial G}{\partial r} = 1/r^2$. Therefore, the first condition must be used to set $q$.

Substituting the equation for $\phi$ into the body BC, we have

$$ \vec U \cdot \hat n(\vec S) + \sum_{j=1}^N \frac{\partial\varphi_j}{\partial n}(\vec S) q_j = 0$$

This boundary condition is linear in $q$ and applies to every point on the body surface. Applying the boundary condition at $N$ specific locations will create $N$ linear equations of the $N$ unknown components of $q$. We will choose the centroid $\vec c$ of each panel to apply this condition. Defining 

$$ a_{i,j} = \frac{\partial\varphi_j}{\partial n}(\vec c_i), \quad b_i = -\vec U \cdot \hat n(\vec c_i) $$

as the components of the influence matrix $A$ and the excitation vector $b$, we have

$$\sum_{j=1}^N a_{i,j} q_j = b_i \quad i=1\ldots N$$

or simply $Aq = b$. So, _how can we determine the correct $q$ for each panel?_ Construct $A,b$ and type

```julia
q = A\b
```

In [ ]:
# dot product of U and `pᵢ.n`
U = [1,0,0]
Uₙ(pᵢ) = U ⋅ pᵢ.n

# derivative of ϕⱼ in direction pᵢ.n
∂ₙϕ(pᵢ,pⱼ) = derivative(t->ϕ(pᵢ.x+t*pᵢ.n,pⱼ),0.)

# Construct A,b
### permutedims(panels) has dimension 1xN
A,b = ∂ₙϕ.(panels,permutedims(panels)),-Uₙ.(panels)

# solve & check that it worked
q = A \ b
A*q ≈ b

## Convergence and validation

We got the solution to the linear system we created, but was that the right system? And how accurate is the solution?

4. _How should we determine if a method is working?_ 

First, lets do a sanity check on the matrix and excitation vector.

In [ ]:
plt1=heatmap(A,yflip=true,colorbar_title="A")
plt2=heatmap(inv.(A),yflip=true,colorbar_title="inv.(A)")
plot(plt1,plt2,layout=(1,2),size=(550,200))

The matrix is diagonally dominated with a value around 6, and the off diagonals get as small as 1/15. Does that make sense?

The diagonal values is the self-induced velocity of the panel in the normal direction. We found earlier that this is exactly $u_n/q = 2\pi$ so the diagonal magnitude looks right.

The off diagonals are the influence of one panel on another. The smallest values will be antipodes, at a distance of $2R$. Then $u_n/q \approx (h/2R)^2 = 1/16$, so this matches as well.

In [ ]:
plot(x,b,xlabel="x/R",ylabel="b/U")

The $b$ vector also looks correct, since $U_n = n_x = x/R$ for the sphere as we set $U=[1,0,0]$. 

---

Next, lets **validate** the method, comparing the numerical solution to a known exact solution.

The analytic potential flow surface velocity on the sphere is

$$ u_\alpha = \frac 32 U \sin(\alpha) $$

where $\alpha$ is the angle of the surface point with respect to the flow direction. 

The numerical solution's velocity is 

$$ \vec u = \vec U + \vec\nabla\phi$$

In [ ]:
# Functions to compute disturbance potential and velocity
φ(x,q,panels) = q'*ϕ.(Ref(x),panels)
∇φ(x,q,panels) = gradient(x->φ(x,q,panels),x)

# Total velocity function
velocity(x,q,panels) = U+∇φ(x,q,panels)

# Get velocity on the panel centroids
### `map` a function to each element of an array
uₐ = map(x->√sum(abs2,velocity(x,q,panels)),panels.x)
α = acos.(b) # acos(n_x)

plot(0:0.01:π,x->1.5sin(x),label="exact",xlabel="α",ylabel="uₐ")
scatter!(α,uₐ,label="$N panels")

Not bad, but there is some error.

**_What type of error is this?_**
 1. System description
 2. Modelling
 3. Truncation
 3. Finite precision
 3. Human

In [ ]:
# Reduce h: set-up -> solve -> measure
panels = sphere(0.3)
A,b = ∂ₙϕ.(panels,permutedims(panels)),-Uₙ.(panels)
q = A \ b
uₐ = map(x->√sum(abs2,velocity(x,q,panels)),panels.x)
α = acos.(b); N = length(α)
scatter!(α,uₐ,label="$N panels")

Excellent. We see **convergence** to the exact solution as we reduce $h$. This confirms we have a small & controllable truncation error. 

We can further quantify this error by computing a relevant integrated quantity. The added mass matrix is a good choice for potential flows, which is defined as 

$$ m_{i,j} = -\rho\oint_S \tilde\phi_i n_j da $$

where $\rho$ is the fluid desity and $\tilde\phi_i$ is the scaled potential resulting from unit velocity in direction $i$. The forces due to an acceleration vector $a$ are then $f = Ma$.

The analytic solution for a sphere is $m_{i,i} = \frac 23 \rho \pi R^3$ on the diagonal and zero for the off diagonals. How does our numerical method perform?

In [ ]:
function added_mass(panels)
    A = ∂ₙϕ.(panels,permutedims(panels))
    B = stack(panels.n)' # all three excitations
    Q = A \ B            # solve for all three q's
    -sum(p->φ(p.x,Q,panels)*p.n'*p.dA,panels)
end
added_mass(panels)/(2π/3) # scale by exact solution

Notice the off diagonals are near machine precision zero. 

Finally lets make a convergence plot with $h$

In [ ]:
using LinearAlgebra: tr # trace of a matrix
sphere_ma_error(h) = 100tr(added_mass(sphere(h)))/2π-100
plot(0.1:0.05:0.5,sphere_ma_error,label=nothing,xlabel="h/R",
        ylabel="trace(M) % error")

We see that the error is $\le 1\%$ for $h/R<0.4$ and converges to zero as $h$ reduces.

## Summary